# Quote identification & text correction

- vérifier si la citation est dans le texte et renvoyer les indices de début et de fin 

    a) citation exacte 

    b) s'il n'y a pas la citation exacte, trouver le passage le plus similaire 
    
- corriger le texte pour qu'il soit lisible

In [ ]:
import pandas as pd
import json 
import re
import math 
import ast 

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken
import time 

import os
from dotenv import load_dotenv
from openai import OpenAI

import pprint

In [ ]:
# gpt 4o mini
load_dotenv()
os.getenv('OPENAI_API_KEY')

client = AsyncOpenAI()
model = "gpt-4o-mini"

In [ ]:
# llama
client = AsyncOpenAI(
    base_url="https://api.scaleway.ai/f1ced67f-44e3-466f-952f-9f6ea4271e45/v1", # Scaleway's Generative APIs service URL
    api_key= os.getenv("SCW_SECRET_KEY") # Your unique API secret key from Scaleway
)
model="llama-3.1-70b-instruct"

In [3]:
def read_parquet(uploaded_file):
    return pd.read_parquet(uploaded_file)

df = read_parquet("//Users/anne/Desktop/Data4Good/ECF GenAI/4_channels_predictions_09_2023_09_2024.parquet")
df.reset_index(inplace=True)
df_exploded = df.explode('claims', ignore_index=True)
df_normalized = pd.json_normalize(df_exploded['claims'])
df_result = df_exploded.drop(columns=['claims']).join(df_normalized)
df_claims = df_result[(df_result['disinformation_score']=='high') & (df_result['pro_anti']!='pro-écologie')]
df_claims

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
67,f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c...,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
87,e57e39657004072e5f998d19e6a8a952ee8f35923b9072...,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
251,5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5c...,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative
259,53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...,2023-09-06 08:38:00,je suis même allé plus loin je vais essayer de...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences"", ""ressou...","[{""keyword"": ""\u00e9coresponsable"", ""timestamp...",2,614,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative
313,6c3bb9299d31332c644b54832399976117d28776c9e22c...,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73775,f875741067fda8011d4aa9ad4c543e52227e31b525fed1...,2024-07-29 13:14:00,sur place voilà ce qu'ils restent par exemple ...,tf1,False,Information - Journal,JT 13h,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""feux"", ""timestamp"": 172225164506...",1,446,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative
74166,cdfed233896d49971897df448465dd43b79ae871b33adb...,2024-08-05 09:16:00,les insectes non on pense que c'est la chaleur...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""g\u00e9othermie"", ""timestamp"": 1...",2,479,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,

## Find quotes 

- checker qu'il n'y a pas d'hallucination
- trouver une méthode pour souligner la quote dans le texte 

In [46]:
df_claims["quote_in_text"] = df_claims.apply(lambda row: row["quote"] in row["text"], axis=1)
# les quote ne sont pas toujours dans les textes -> mesure de similarité entre claim et quote ? 
df_claims.groupby('quote_in_text').count()['id']

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_5363/2088228417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims["quote_in_text"] = df_claims.apply(lambda row: row["quote"] in row["text"], axis=1)


quote_in_text
False    908
True     546
Name: id, dtype: int64

In [47]:
not_exact = df_claims[df_claims.quote_in_text == False]

In [56]:
not_exact['m_quote'] = not_exact['quote'].apply(lambda s: re.sub(r'[^\w\s]', '', s))

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_5363/3782613564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_exact['m_quote'] = not_exact['quote'].apply(lambda s: re.sub(r'[^\w\s]', '', s))


In [57]:
not_exact["m_quote_in_text"] = not_exact.apply(lambda row: row["m_quote"] in row["text"], axis=1)
# les quote ne sont pas toujours dans les textes -> mesure de similarité entre claim et quote ? 
not_exact.groupby('m_quote_in_text').count()['id']

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_5363/2620873879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_exact["m_quote_in_text"] = not_exact.apply(lambda row: row["m_quote"] in row["text"], axis=1)


m_quote_in_text
False    888
True      20
Name: id, dtype: int64

In [58]:
not_exact_2 = not_exact[not_exact.m_quote_in_text == False]

In [ ]:
# extract main words from the quote and see if it is in the text 

In [59]:
pprint.pprint(not_exact_2.iloc[0].quote)
pprint.pprint(not_exact_2.iloc[0].claim)
pprint.pprint(not_exact_2.iloc[0].text)

('Le réchauffement climatique est causé par une élite qui contrôle les médias '
 'pour imposer un agenda eco-responsable.')
('Le réchauffement climatique est causé par une élite qui contrôle les médias '
 'pour imposer un agenda eco-responsable.')
('je suis même allé plus loin je vais essayer de me mettre à sa place vous '
 'laisser de vous mettre à la place de manuel bompard un moment dimitri il '
 'faut passer par là parce que si vous voulez faire avancer le vivre ensemble '
 'il faut essayer de comprendre les gens plutôt que de les juger je crois que '
 'ça fait pas du bien à la canicule mais pour répondre à votre question oui ce '
 "matin j'essaie de m'imaginer l'état d'esprit notre invité quand ils bien ici "
 "bon alors racontait alors il faut l'imaginée quand il est en approche de "
 "notre immeuble c'est-à-dire ben il est la marche sur les quais de seine avec "
 "son attaché de presse un moment il s'arrête il lève la tête yvonne la autres "
 "dent immeubles ira et là dedans eur

In [67]:
# quantifier la similitude entre la quote et le texte 
#not_exact_2
df_claims.loc[259]

id                         53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...
start                                                    2023-09-06 08:38:00
text                       je suis même allé plus loin je vais essayer de...
channel_name                                                         europe1
channel_is_radio                                                        True
channel_program_type                                  Information - Magazine
channel_program                                               Europe 1 Matin
themes                     ["changement_climatique_consequences", "ressou...
keywords                   [{"keyword": "\u00e9coresponsable", "timestamp...
num_keywords                                                               2
num_tokens                                                               614
analysis                   Cette allégation frôle la théorie du complot, ...
claim                      Le réchauffement climatique est causé par une ...

In [68]:
pprint.pprint(df_claims.loc[259].quote)
pprint.pprint(df_claims.loc[259].claim)
pprint.pprint(df_claims.loc[259].text)

('Le réchauffement climatique est causé par une élite qui contrôle les médias '
 'pour imposer un agenda eco-responsable.')
('Le réchauffement climatique est causé par une élite qui contrôle les médias '
 'pour imposer un agenda eco-responsable.')
('je suis même allé plus loin je vais essayer de me mettre à sa place vous '
 'laisser de vous mettre à la place de manuel bompard un moment dimitri il '
 'faut passer par là parce que si vous voulez faire avancer le vivre ensemble '
 'il faut essayer de comprendre les gens plutôt que de les juger je crois que '
 'ça fait pas du bien à la canicule mais pour répondre à votre question oui ce '
 "matin j'essaie de m'imaginer l'état d'esprit notre invité quand ils bien ici "
 "bon alors racontait alors il faut l'imaginée quand il est en approche de "
 "notre immeuble c'est-à-dire ben il est la marche sur les quais de seine avec "
 "son attaché de presse un moment il s'arrête il lève la tête yvonne la autres "
 "dent immeubles ira et là dedans eur

In [80]:
df_claims.loc[259].text.split('.')

["je suis même allé plus loin je vais essayer de me mettre à sa place vous laisser de vous mettre à la place de manuel bompard un moment dimitri il faut passer par là parce que si vous voulez faire avancer le vivre ensemble il faut essayer de comprendre les gens plutôt que de les juger je crois que ça fait pas du bien à la canicule mais pour répondre à votre question oui ce matin j'essaie de m'imaginer l'état d'esprit notre invité quand ils bien ici bon alors racontait alors il faut l'imaginée quand il est en approche de notre immeuble c'est-à-dire ben il est la marche sur les quais de seine avec son attaché de presse un moment il s'arrête il lève la tête yvonne la autres dent immeubles ira et là dedans europe et un europe un le jdd cnews c'est la tour du mordor l'âge l'observer jouer bien plus il approchait plus le fardeau était lourd je souffrais et comme vous le voyez je suivais aux jumelles du bureau de lagardère aux jumelles depuis le bureau du papa auch voyait sa souffrance je dé

In [ ]:
# combien de phrases par quote
not_exact_2.quote.apply(lambda t: )

259      Le réchauffement climatique est causé par une ...
313      ...c'était hier soir dans cette à vous sur fra...
444      les climatologues ne croient plus leurs termes...
607      Les canicules sont des événements normaux et n...
608      Il n'y a pas de lien entre la hausse des tempé...
                               ...                        
73449         le dérèglement climatique phénomène naturel.
73531    nous on partirait plutôt sur de la capture eff...
73690    Les bénéfices de fruits et légumes frais n'ont...
74219    Pour l'instant, le risque de chutes de séracs ...
74850      Cette astuce elle est fausse et voilà pourquoi.
Name: quote, Length: 888, dtype: object

In [70]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/anne/Desktop/Data4Good/climateguard/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/anne/Desktop/Data4Good/climateguard/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/anne/Desktop/Data4Good/climateguard/.venv/lib/python3.12/site-packages/ipykernel/

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Problème avec le sentence embedding : le texte n'a pas de ponctuation 

In [71]:
quote = df_claims.loc[259].quote
text = df_claims.loc[259].text

# Diviser le texte en segments (phrases)
segments = text.split(". ")

# Encoder la citation et les segments
quote_embedding = model.encode(quote, convert_to_tensor=True)
segment_embeddings = model.encode(segments, convert_to_tensor=True)

# Calculer les similarités cosinus
similarities = util.pytorch_cos_sim(quote_embedding, segment_embeddings)

# Trouver le segment le plus proche
most_similar_idx = similarities.argmax()
most_similar_segment = segments[most_similar_idx]
similarity_score = similarities[0][most_similar_idx].item()

# Résultat
if similarity_score > 0.8:  # Ajustez le seuil selon vos besoins
    print(f"Match trouvé : '{most_similar_segment}' avec un score de similarité de {similarity_score:.2f}")
else:
    print("Aucun passage significativement similaire trouvé.")

Aucun passage significativement similaire trouvé.


In [ ]:
Class Find

def closest_segment(text, quote):
    segments = text.split(". ")

    # Encoder la citation et les segments
    quote_embedding = model.encode(quote, convert_to_tensor=True)
    segment_embeddings = model.encode(segments, convert_to_tensor=True)

    # Calculer les similarités cosinus
    similarities = util.pytorch_cos_sim(quote_embedding, segment_embeddings)

    # Trouver le segment le plus proche
    most_similar_idx = similarities.argmax()
    most_similar_segment = segments[most_similar_idx]
    similarity_score = similarities[0][most_similar_idx].item()

    # Résultat
    if similarity_score > 0.8:  # Ajustez le seuil selon vos besoins
        print(f"Match trouvé : '{most_similar_segment}' avec un score de similarité de {similarity_score:.2f}")
    else:
        print("Aucun passage significativement similaire trouvé.")

In [74]:
not_exact_2.iloc[1].m_quote in not_exact_2.iloc[1].text

False

In [81]:
from difflib import HtmlDiff

text1 = not_exact_2.iloc[1].m_quote
text2 = not_exact_2.iloc[1].text

html_diff = HtmlDiff()
html_result = html_diff.make_file(text1.splitlines(), text2.splitlines())

# Sauvegarder le résultat dans un fichier HTML
with open("diff.html", "w") as file:
    file.write(html_result)

print("Différence générée dans 'diff.html'")

Différence générée dans 'diff.html'


In [124]:
from sentence_transformers import SentenceTransformer, util

def sliding_window(text, window_size, stride):
    """
    Divise le texte en segments de taille fixe avec un saut défini.
    
    Args:
    - text (str): Texte à diviser.
    - window_size (int): Nombre de mots par segment.
    - stride (int): Saut entre les segments.
    
    Returns:
    - List[str]: Liste de segments.
    """
    words = text.split()
    return [
        " ".join(words[i:i+window_size])
        for i in range(0, len(words) - window_size + 1, stride)
    ]

def find_most_similar_segments(df, model_name='paraphrase-MiniLM-L6-v2', pruning_threshold = 0.7):
    """
    Trouve le passage le plus similaire pour chaque ligne d'un DataFrame.
    
    Args:
    - df (pd.DataFrame): DataFrame contenant 'quote' et 'text'.
    - model_name (str): Modèle de SentenceTransformer.
    - stride (int): Saut entre fenêtres (en mots).
    
    Returns:
    - pd.DataFrame: DataFrame enrichi avec les résultats.
    """
    # Charger le modèle
    model = SentenceTransformer(model_name)
    
    results = []
    
    for idx, row in df.iterrows():
        quote = row['m_quote']
        text = row['text']
        window_size = int(1 * len(quote.split()))
        stride =  int(0.25 * len(quote.split()))
        
        # Générer des fenêtres glissantes
        segments = sliding_window(text, window_size, stride)
        
        # Encoder la quote et les segments
        quote_embedding = model.encode(quote, convert_to_tensor=True)

        best_similarity = -1
        
        # Processus avec pruning
        for segment in segments:
            segment_embedding = model.encode(segment, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(quote_embedding, segment_embedding).item()
            
            # Si on trouve une meilleure correspondance
            if similarity > best_similarity:
                best_similarity = similarity
                best_segment = segment
                start_index = text.find(segment)
                end_index = start_index + len(segment)
            
            # Arrêter si le seuil de pruning est atteint
            if best_similarity >= pruning_threshold:
                break
        
        # Stocker les résultats
        results.append({
            "quote": quote,
            "most_similar_segment": best_segment,
            "start_index": start_index,
            "end_index": end_index,
            "similarity_score": similarity,
        })
    
    # Ajouter les résultats au DataFrame
    #result_df = pd.DataFrame(results)
    return results #pd.concat([df.reset_index(drop=True), result_df], axis=1)

result_df = find_most_similar_segments(not_exact_2.iloc[[1]])

In [125]:
result_df

[{'quote': 'cétait hier soir dans cette à vous sur france cinq nicolas sarkozy interrogé sur cette stupide polémique',
  'most_similar_segment': "petit dernier pour la route c'était hier soir dans cette à vous sur france cinq nicolas sarkozy",
  'start_index': 172,
  'end_index': 267,
  'similarity_score': 0.789903998374939}]

In [107]:
not_exact_2.iloc[1]['text']

"pour faire réussir nos enfants dans quel monde on est dans deux mille vingt et un avec de l'éco anxiété avec avec un changement climatique ils pèsent sur nos vies avait un petit dernier pour la route c'était hier soir dans cette à vous sur france cinq nicolas sarkozy interrogé sur cette stupide polémique sur ses généreux donc de quelques-uns en faveur des restos du c'cur est la mission très aussi pour l'ancien président françois hollande se prend une balle perdue c'est la différence entre landes et moi et vous les moins de riches moi je terminer veux en moins merci de beaucoup pauvres zéro c'mme est begley terminé c'était merci votre beaucoup zapping politique zéro tous les matins sur europe un exercice je sais que ça vous plaît chers auditeurs vous hésitez pas vous réagissez sur les réseaux sociaux et on vous attend aussi à l'antenne zéro à quatre vingt trente neuf vingt et un jérôme à la semaine prochaine à la semaine prochaine voilà il est huit heures cinquante six l'heure de retro

In [ ]:
quote[:len(quote.split()[:5])]

'cétai'

In [122]:
# trouver l'indice des premiers mots 
#aux_df = not_exact_2.iloc[[1]]
text = not_exact_2.iloc[1]['text']
quote = not_exact_2.iloc[1]['quote']

start_index = text.find(quote[:sum([len(v) for v in quote.split()[:5]])])
end_index = start_index + len(quote)

In [123]:
quote[:sum([len(v) for v in quote.split()[:5]])]

"...c'était hier soir dans c"

In [121]:
text

"pour faire réussir nos enfants dans quel monde on est dans deux mille vingt et un avec de l'éco anxiété avec avec un changement climatique ils pèsent sur nos vies avait un petit dernier pour la route c'était hier soir dans cette à vous sur france cinq nicolas sarkozy interrogé sur cette stupide polémique sur ses généreux donc de quelques-uns en faveur des restos du c'cur est la mission très aussi pour l'ancien président françois hollande se prend une balle perdue c'est la différence entre landes et moi et vous les moins de riches moi je terminer veux en moins merci de beaucoup pauvres zéro c'mme est begley terminé c'était merci votre beaucoup zapping politique zéro tous les matins sur europe un exercice je sais que ça vous plaît chers auditeurs vous hésitez pas vous réagissez sur les réseaux sociaux et on vous attend aussi à l'antenne zéro à quatre vingt trente neuf vingt et un jérôme à la semaine prochaine à la semaine prochaine voilà il est huit heures cinquante six l'heure de retro

In [118]:
text[start_index:end_index]

''

In [89]:
not_exact_2.iloc[[1]]

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,...,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,quote_in_text,m_quote,m_quote_in_text
313,6c3bb9299d31332c644b54832399976117d28776c9e22c...,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative,False,cétait hier soir dans cette à vous sur france ...,False


In [73]:
pprint.pprint(not_exact_2.iloc[1].m_quote)
pprint.pprint(not_exact_2.iloc[1].claim)
pprint.pprint(not_exact_2.iloc[1].text)

('cétait hier soir dans cette à vous sur france cinq nicolas sarkozy interrogé '
 'sur cette stupide polémique')
('Le changement climatique est traité comme une simple polémique par des '
 'personnalités politiques.')
('pour faire réussir nos enfants dans quel monde on est dans deux mille vingt '
 "et un avec de l'éco anxiété avec avec un changement climatique ils pèsent "
 "sur nos vies avait un petit dernier pour la route c'était hier soir dans "
 'cette à vous sur france cinq nicolas sarkozy interrogé sur cette stupide '
 'polémique sur ses généreux donc de quelques-uns en faveur des restos du '
 "c'cur est la mission très aussi pour l'ancien président françois hollande se "
 "prend une balle perdue c'est la différence entre landes et moi et vous les "
 'moins de riches moi je terminer veux en moins merci de beaucoup pauvres zéro '
 "c'mme est begley terminé c'était merci votre beaucoup zapping politique zéro "
 'tous les matins sur europe un exercice je sais que ça vous plaît chers

In [ ]:
# diff entre les deux textes


In [ ]:
# remove 
res = re.sub(r'[^\w\s]', '', not_exact.iloc[0].quote)
res

'il y a par exemple le giec transhumaniste qui trimballe un pantalon chaise'

In [51]:
pprint.pprint(not_exact.iloc[0].quote)
pprint.pprint(not_exact.iloc[0].claim)
pprint.pprint(not_exact.iloc[0].text)

'il y a par exemple le giec transhumaniste qui trimballe un pantalon chaise.'
'La mention du GIEC comme un "transhumaniste".'
("salle d'attente il y a toutes les classes rageant tous les âges toutes les "
 'origines vraiment tout dire tout le monde dans cette salle vous avez une '
 "question euh dimitris c'est terrible c'est le caucus de l'histoire ces ronal "
 "que vous venez d'entendre il n'est pas au bout de ses surprises à ce "
 "moment-là ils ignorent totalement qu'il est victime d'une gigantesque "
 'caméras cachées qui a duré douze jours mêlant se moque de lui en regardant '
 "la série non justement c'est hilarant parce que les situations sont "
 'improbables déjà les jurés sont confinés pendant dix jours dans dix un jours '
 "hôtel dans ce un qui hôtel n'ce arrivent jamais qui dans n'la arrivent "
 'réalité jamais dans la réalité et on sait que le procès en question oppose '
 "la directrice d'une marque de vêtements à un ouvrier qui auraient uriné sur "
 "ces t-shirts à l'usine

In [ ]:
# comparaison de deux textes 
from difflib import HtmlDiff

text1 = """Bonjour, je m'appelle ChatGPT."""
text2 = """Bonjour, je m'appelle ChatGPT, enchanté !"""

html_diff = HtmlDiff()
html_result = html_diff.make_file(text1.splitlines(), text2.splitlines())

# Sauvegarder le résultat dans un fichier HTML
with open("diff.html", "w") as file:
    file.write(html_result)

print("Différence générée dans 'diff.html'")

In [62]:
df_claims.loc[87]

id                         e57e39657004072e5f998d19e6a8a952ee8f35923b9072...
start                                                    2023-09-04 07:50:00
text                       salle d'attente il y a toutes les classes rage...
channel_name                                                         europe1
channel_is_radio                                                        True
channel_program_type                                  Information - Magazine
channel_program                                               Europe 1 Matin
themes                     ["changement_climatique_causes", "changement_c...
keywords                   [{"keyword": "giec", "timestamp": 169380664205...
num_keywords                                                               2
num_tokens                                                               554
analysis                   Cette allégation semble utiliser le terme 'tra...
claim                          La mention du GIEC comme un "transhumaniste".

In [ ]:
system_prompt_v0 = '''
    L'utilisateur fournira un extrait de 2 minutes d'une émission de télévision ou de radio, accompagné d'une citation tirée de cette transcription.
    La transcription sera phonétique, sans ponctuation, et pourra contenir des erreurs (vocabulaire incorrect, mauvais découpage).
    
    Tâche principale : 
        Corrige le texte pour le rendre en français correct, en reformulant les phrases lorsqu'elles n'ont pas de sens, tout en respectant la phonétique.
        Si des nombres apparaissent sous forme de texte, transforme-les en chiffres (par exemple, "quatre-vingt" devient 80).
        Crée des paragraphes lorsque plusieurs sujets sont abordés dans le même transcript.
    
    Exceptions :
        Dans le texte, ne modifie pas la citation fournie par l'utilisateur, même si elle comporte des erreurs. La citation (quote) doit rester identique à ce qui est donné.
        
    Exemple :
        Text : elles doivent donc être renouvelés et sané col débuts en europe. 
        Quote : elles doivent donc être renouvelés
        Correction : elles doivent donc être renouvelés, et ce n'est que le début en Europe.

        Tu ne retournes que la correction, contenant la citation identique à ce qui a été fourni. 

    Rappel : Concentre-toi sur la correction de la transcription tout en maintenant la citation intacte et en appliquant les règles concernant les nombres.
'''

In [64]:
system_prompt = '''
    L'utilisateur fournira un extrait de 2 minutes d'une émission de télévision ou de radio accompagné d'une citation tirée de la transcription.

    **Tâche principale** :
    1. Corrigez la transcription pour la rendre en français correct, en reformulant les phrases pour leur donner du sens si nécessaire, tout en respectant la phonétique.
    4. Supprime les répétitions inutiles (ex. "l'vu ai je vu l'je ai l'vu ai" devient "je l'ai vu").
    2. Transformez les nombres écrits en toutes lettres dans le texte en chiffres (ex. "quatre-vingt" devient 80).
    3. Divisez le texte en paragraphes si plusieurs sujets sont abordés.

    **Exception** :
    - Ne modifiez pas la citation fournie par l'utilisateur, même si elle contient des erreurs. La citation (quote) doit rester identique dans le texte corrigé.

    **Exemple** :
    - Texte : elles doivent donc être renouvelés et sané col débuts en europe.  
    - Citation : elles doivent donc être renouvelés  
    - Correction : elles doivent donc être renouvelés, et ce n'est que le début en Europe.

    Retournez uniquement le texte corrigé, avec la citation inchangée.
'''

correction = await correct_text(df_claims.loc[87].text, df_claims.loc[87].claim)
pprint.pprint(correction)

("Salle d'attente, il y a toutes les classes, toutes les origines, vraiment "
 'tout le monde dans cette salle. Vous avez une question, euh, Dimitris ? '
 "C'est terrible, c'est le cauchemar de l'histoire. Ce Ronald que vous venez "
 "d'entendre n'est pas au bout de ses surprises. À ce moment-là, ils ignorent "
 "totalement qu'il est victime d'une gigantesque caméra cachée qui a duré 12 "
 "jours. Mêlant se moque de lui en regardant la série, non, justement, c'est "
 'hilarant parce que les situations sont improbables. Déjà, les jurés sont '
 "confinés pendant 10 jours dans un hôtel, ce qui n'arrive jamais dans la "
 "réalité. Et on sait que le procès en question oppose la directrice d'une "
 'marque de vêtements à un ouvrier qui aurait uriné sur ces t-shirts à '
 "l'usine. Toutes les personnages sont sympas mais très bizarres. Il y a, par "
 'exemple, le GIEC transhumaniste qui trimballe un pantalon-chaise qui lui '
 "permet de s'asseoir quand il le souhaite. Si l'avocat qui n'a pas b

In [ ]:
correction

"salle d'attente il y a toutes les classes rageant tous les âges toutes les origines vraiment tout dire tout le monde dans cette salle vous avez une question euh dimitris c'est terrible c'est le caucus de l'histoire ces ronal que vous venez d'entendre il n'est pas au bout de ses surprises à ce moment-là ils ignorent totalement qu'il est victime d'une gigantesque caméras cachées qui a duré douze jours mêlant se moque de lui en regardant la série non justement c'est hilarant parce que les situations sont improbables déjà les jurés sont confinés pendant dix jours dans dix un jours hôtel dans ce un qui hôtel n'ce arrivent jamais qui dans n'la arrivent réalité jamais dans la réalité et on sait que le procès en question oppose la directrice d'une marque de vêtements à un ouvrier qui auraient uriné sur ces t-shirts à l'usine toutes les personnages sont sympas mais très bizarre il y a par exemple le giec transhumaniste qui trimballe un pantalon chaise qui lui permet de s'asseoir quand il se pr

In [7]:
contradictions = df_claims[df_claims.contradiction != 'N/A']

In [14]:
async def correct_text(
        text: str, quote: str, system_prompt: str=system_prompt, tpm_limit: int=2e6
    ):
    content = "Texte: {}, Citation: {}" .format(text, quote)
    if model=="gpt-4o-mini":
        encoding = tiktoken.encoding_for_model(model)
        prompt_tokens = len(encoding.encode(system_prompt))
        text_tokens = len(encoding.encode(content))
        total_tokens = prompt_tokens + text_tokens
        wait_time_s = 60 * total_tokens / tpm_limit
        await asyncio.sleep(wait_time_s)
    else:
        await asyncio.sleep(60 * 50 / tpm_limit)
    
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

async def run_correction(texts: list[str], quotes: list[str]):
    semaphore = asyncio.Semaphore(10)
    
    async def bounded_detect_claim(text, quote):
        async with semaphore:
            return await correct_text(text, quote)
    
    correction = await tqdm.gather(*[bounded_detect_claim(text, quote) for text, quote in zip(texts, quotes)])
    return correction

In [22]:
contradictions.iloc[:1].quote

883    Les déplacements en voiture sont la seule solu...
Name: quote, dtype: object

In [27]:
contradictions.iloc[:2].quote.values

array(['Les déplacements en voiture sont la seule solution viable pour la mobilité.',
       'on ne peut pas faire le lien directement avec le réchauffement'],
      dtype=object)

In [45]:
t = "l'effet de levier est un concept utilisé en comptabilité "
t.strip()

"l'effet de levier est un concept utilisé en comptabilité"

In [41]:
correction = await correct_text(contradictions.iloc[:10].text, contradictions.iloc[:10].quote)
pprint.pprint(correction)

('Voici la correction du texte :\n'
 '\n'
 '883     La conserve de 385 grammes...\n'
 '1812    Le réchauffement climatique, la collecte de Grenoble...\n'
 "2612    Oui, j'utilise des pesticides, alors pour savoir...\n"
 '2728    Littoraux, donc pour trouver du soleil, vous allez...\n'
 '2778    Réseaux sociaux, et je vous vois piloter des hélicoptères...\n'
 '2878    Six seize, les actions qui ont commencé à vérifier...\n'
 '3125    Le lendemain, et le lendemain du surlendemain...\n'
 '3167    Font pour nous, pour nous spécialement, pauvres types...\n'
 "4004    Et aujourd'hui, on se retrouve à construire des...\n"
 "4072    Ce fameux pot de yaourt, pour savoir ce qu'il devient...\n"
 '\n'
 "Notez que la citation fournie n'a pas été modifiée, même si elle comporte "
 'des erreurs. La citation doit rester identique à ce qui est donné.')


In [ ]:
correction = await correct_text(contradictions.iloc[:10].text.values, contradictions.iloc[:10].quote.values)
pprint.pprint(correction)

('Voici les corrections du texte :\n'
 '\n'
 '"La conserve de 385 grammes coûte 2,23 euros le kilo. Délicieuse, elle est '
 "égale à délicieux pour votre santé. Bougez plus, à l'intention de notre très "
 'chère cliente, propriétaire de la Toyota AGO Cross. Vous pouvez la déplacer '
 "vers les caméras de surveillance, car j'aimerais bien l'admirer de plus près "
 "sur les écrans. Merci d'avance. Découvrez la Toyota Yaris Cross, le "
 'crossover urbain, à partir de 119 euros par mois. Entretien clé Toyota Yaris '
 'Cross dynamique LLD, 37 mois, 30 000 km. Premier loyer, 3 969 euros. Cent '
 'réservée soixante-dix aux euros particuliers, réservée aux particuliers sous '
 "conditions de reprise et commande jusqu'au 25 septembre. Détaillez sur "
 'Toyota.fr. Portes ouvertes ce week-end. Pensez à covoiturer. Écoutez Europe '
 '1 partout, dès maintenant en DAB+, entre Paris et Marseille sans changer de '
 "fréquence, et bientôt dans toute la France. Europe 1, toute l'actualité, "
 'débat, cé

In [34]:
contradictions.iloc[:1].text.values

array(["la conserve de trois cent quatre-vingt-cinq grammes c six ht soit deux euros vingt trois le kilo délicieuse au égale délicieux pour votre santé bougez plus a l'intention de notre très chère clientes propriétaire de la toyota ago crosses vous pouvez la déplacer vers les caméras de surveillance parce que j'aimerais bien admirer de plus près sur les écrans merci d'avance découvrez la toyota yaris cross le crossover urbain à partir de cent dix-neuf euros par mois entretien clé toyota yaris cross dynamique lld trente-sept mois trente mille km premier loyer premier trois loyer mille trois neuf cent mille soixante neuf euros cent réservée soixante-dix aux euros particuliers réservée aux particuliers sous conditions de reprise et commander jusqu'au vingt-cinq septembre détaille sur toyota point fr portes ouvertes ce week-end pensez à covoiturer écouter europe un partout dès maintenant en dab plus entre paris et marseille sans changer de fréquence et bientôt dans toute la france europe 

In [30]:
correction

'Les déplacements en voiture sont la seule solution viable pour la mobilité. Cependant, il est important de prendre en compte les impacts environnementaux négatifs des voitures et leur contribution aux émissions de gaz à effet de serre. Des alternatives de transport comme le vélo et les transports en commun sont souvent plus viables et devraient être considérées.'

In [35]:
pprint.pprint(correction)

('La conserve de 385 grammes coûte 6,23 euros le kilo. Délicieuse, elle est '
 "égale à délicieux pour votre santé. Bougez plus, à l'intention de notre très "
 'chère cliente, propriétaire de la Toyota Ago Cross. Vous pouvez la déplacer '
 "vers les caméras de surveillance, parce que j'aimerais bien l'admirer de "
 "plus près sur les écrans. Merci d'avance. Découvrez la Toyota Yaris Cross, "
 'le crossover urbain, à partir de 119 euros par mois, entretien clé en main. '
 'Toyota Yaris Cross Dynamique LLD, 37 mois, 30 000 km, premier loyer 3 969 '
 'euros, cent réservée soixante-dix euros aux particuliers, réservée aux '
 "particuliers sous conditions de reprise et commande jusqu'au 25 septembre. "
 'Détaillez sur toyota.fr. Portes ouvertes ce week-end. Pensez à covoiturer. '
 'Écoutez Europe 1 partout, dès maintenant en DAB+, entre Paris et Marseille '
 'sans changer de fréquence, et bientôt dans toute la France. Europe 1, toute '
 "l'actualité, débat, cédant, punchline, à tout de suit

In [20]:
contradictions.iloc[:1].text

883    la conserve de trois cent quatre-vingt-cinq gr...
Name: text, dtype: object

In [64]:
correction = await run_correction(contradictions.iloc[:10].text, contradictions.iloc[:10].quote)

  0%|          | 0/10 [00:00<?, ?it/s]

Texte: littoraux donc pour trouver du soleil vous allez au sud exactement demain au sud d'une ligne bordeaux aurillac lyon un ciel généralement clairs voilés mais en tout cas une atmosphère agréable ensoleillée avec juste du vent d'autan en plaine toulousaine jusqu'à cinquante km en rafale et puis donc les températures contrasté demain entre treize et dix-sept au nord sous la pluie sept à vingt-deux voire vingt-quatre dans le sud sous le soleil vingt-quatre ce sera pour bastia vingt-deux à marseille et nice vingt à bordeaux et toulouse il fera dix-sept à nantes seize à paris quinze à lille et strasbourg treize à londres ça reste quand même doux pour la saison saison merci anthony kaczmarek chez aldi nos pains et viennoiseries sont cuits dans nos magasins tout au long de la journée tous nos engagements qualité sur aldi point fr aldi place un nouveau consommateur pour votre santé bougez plus dix-neuf heures sur europe un dans un instant les gta les grands témoins de l'actualité du vendre

100%|██████████| 10/10 [00:08<00:00,  1.20it/s]


In [ ]:
# with gpt-4o-mini
start_time = time.time()
subset = contradictions.iloc[:10]
correction = await run_correction(subset.text, subset.quote)

elapsed_time = time.time() - start_time

print(f"Time taken for {len(subset)} items: {elapsed_time:.2f} seconds")

  0%|          | 0/10 [00:00<?, ?it/s]

Texte: six seize les actions qui ont commencé à vérifier autour de soi s'il y a des problèmes particuliers et puis surtout comme il a été dit éviter de circuler à partir de mercredi la nuit de mercredi à jeudi et jeudi matin sur un quart nord-ouest avec les vacances avec ce jour férié au milieu on ne sait que les jeunes son bon vouloir se déplacer c'est normal et les mesures de sécurité et d'anticipation alix roumagnac président de predict services avec nous en en direct dans europe europe un trésor un depuis trésor depuis xynthia en deux mille dix e on a fait beaucoup de progrès vous le constatez vous qui travaillez pour les communes les collectivités en termes de prévention de préparation d'anticipation beaucoup de beaucoup de choses ont évolué déjà c'est là la généralisation de ce qu'on appelle les plans communaux de sauvegarde c'est-à-dire que chaque commune a réalisé un plan d'action pour chaque type d'aléas inondations tempêtes submersions marines des des actions sont identifiés 

100%|██████████| 10/10 [00:10<00:00,  1.08s/it]

Time taken for 10 items: 10.86 seconds


In [ ]:
# with llama
start_time = time.time()
subset = contradictions.iloc[:10]
correction = await run_correction(subset.text, subset.quote)

elapsed_time = time.time() - start_time

print(f"Time taken for {len(subset)} items: {elapsed_time:.2f} seconds")

  0%|          | 0/10 [00:00<?, ?it/s]

In [78]:
correction

["La conserve de 385 grammes c'est 6 HT soit 2,23 euros le kilo, délicieuse, égale à délicieux pour votre santé. Bougez plus à l'intention de notre très chère cliente, propriétaire de la Toyota Yaris Cross. Vous pouvez la déplacer vers les caméras de surveillance parce que j'aimerais bien admirer de plus près sur les écrans, merci d'avance. Découvrez la Toyota Yaris Cross, le crossover urbain à partir de 119 euros par mois. Entretien clé Toyota Yaris Cross dynamique LLD 37 mois, 30 000 km, premier loyer, premier trois loyers, 1 369 euros, réservée aux particuliers sous conditions de reprise et commande jusqu'au 25 septembre. Détails sur toyota.fr. Portes ouvertes ce week-end, pensez à covoiturer. Écoutez Europe 1 partout dès maintenant en DAB+, entre Paris et Marseille sans changer de fréquence et bientôt dans toute la France. Europe 1, toute l'actualité, débat, cédant punchline, à tout de suite sur Europe 1. Klein, 18 heures, 19 heures, Laurence Ferrari sur Europe 1. Nouvelle voiture,

In [66]:
len(correction)

10

In [73]:
pprint.pprint(correction[0])

('Correction : La conserve de 385 grammes coûte 6 HT, soit 2,23 euros le kilo. '
 'Délicieuse, elle est également délicieuse pour votre santé. Bougez plus ! À '
 "l'intention de notre très chère cliente, propriétaire de la Toyota Yaris "
 'Cross, vous pouvez la déplacer vers les caméras de surveillance, parce que '
 "j'aimerais bien admirer de plus près sur les écrans. Merci d'avance. "
 'Découvrez la Toyota Yaris Cross, le crossover urbain, à partir de 119 euros '
 'par mois. Entretien clé Toyota Yaris Cross dynamique LLD 37 mois, 30 000 km, '
 'premier loyer, premier trois loyers, 1 369 euros. Réservé aux particuliers, '
 "sous conditions de reprise, et commandez jusqu'au 25 septembre. Détails sur "
 'toyota.fr. Portes ouvertes ce week-end, pensez à covoiturer. Écoutez Europe '
 '1 partout dès maintenant en DAB+, entre Paris et Marseille, sans changer de '
 "fréquence, et bientôt dans toute la France. Europe 1, toute l'actualité, "
 'débat, cédant punchline. À tout de suite sur Europ

In [72]:
contradictions.iloc[:10].text.iloc[0]

"la conserve de trois cent quatre-vingt-cinq grammes c six ht soit deux euros vingt trois le kilo délicieuse au égale délicieux pour votre santé bougez plus a l'intention de notre très chère clientes propriétaire de la toyota ago crosses vous pouvez la déplacer vers les caméras de surveillance parce que j'aimerais bien admirer de plus près sur les écrans merci d'avance découvrez la toyota yaris cross le crossover urbain à partir de cent dix-neuf euros par mois entretien clé toyota yaris cross dynamique lld trente-sept mois trente mille km premier loyer premier trois loyer mille trois neuf cent mille soixante neuf euros cent réservée soixante-dix aux euros particuliers réservée aux particuliers sous conditions de reprise et commander jusqu'au vingt-cinq septembre détaille sur toyota point fr portes ouvertes ce week-end pensez à covoiturer écouter europe un partout dès maintenant en dab plus entre paris et marseille sans changer de fréquence et bientôt dans toute la france europe un tout

In [65]:
correction

["Correction : La conserve de 385 grammes coûte 6 HT, soit 2,23 euros le kilo. Délicieuse, elle est également délicieuse pour votre santé. Bougez plus ! À l'intention de notre très chère cliente, propriétaire de la Toyota Yaris Cross, vous pouvez la déplacer vers les caméras de surveillance, parce que j'aimerais bien admirer de plus près sur les écrans. Merci d'avance. Découvrez la Toyota Yaris Cross, le crossover urbain, à partir de 119 euros par mois. Entretien clé Toyota Yaris Cross dynamique LLD 37 mois, 30 000 km, premier loyer, premier trois loyers, 1 369 euros. Réservé aux particuliers, sous conditions de reprise, et commandez jusqu'au 25 septembre. Détails sur toyota.fr. Portes ouvertes ce week-end, pensez à covoiturer. Écoutez Europe 1 partout dès maintenant en DAB+, entre Paris et Marseille, sans changer de fréquence, et bientôt dans toute la France. Europe 1, toute l'actualité, débat, cédant punchline. À tout de suite sur Europe 1, Klein, 18 heures, 19 heures, Laurence Ferra

In [62]:
corr = await correct_text("Les déplacement en voiture sont la seule solution", "Les déplacement")
corr

Texte: {text}, Citation: {quote}


"Je suis désolé, mais il semble que vous n'ayez pas fourni de texte ou de citation. Pourriez-vous s'il vous plaît les partager afin que je puisse vous aider ?"

In [59]:
contradictions.iloc[:10].quote

883     Les déplacements en voiture sont la seule solu...
1812    on ne peut pas faire le lien directement avec ...
2612    techniquement il n'y a pas de résidus si vous ...
2728    la voiture c'est meilleur pour la qualité de l...
2778    un jeu oui je crois que j'aime je crois que je...
2878    il y a il a aucun lien avec le dérèglement cli...
3125    la c'voiture est meilleur pour la qualité de l...
3167                                     on a aucune aide
4004    Les éoliennes sont très laides et peu efficace...
4072    Les pots de yaourt sont souvent envoyés à l'ét...
Name: quote, dtype: object

In [56]:
correction

["Il semble que vous n'ayez pas fourni de texte ou de citation. Veuillez soumettre l'extrait de l'émission ainsi que la citation que vous souhaitez conserver, et je me ferai un plaisir de vous aider à corriger le texte.",
 "Il semble que vous n'ayez pas fourni de texte ou de citation. Veuillez soumettre l'extrait de l'émission ainsi que la citation que vous souhaitez conserver, et je me ferai un plaisir de vous aider à corriger le texte.",
 "Il semble que vous n'ayez pas fourni de texte ou de citation. Veuillez soumettre l'extrait de l'émission ainsi que la citation que vous souhaitez que je corrige.",
 "Il semble que vous n'ayez pas fourni de texte ou de citation. Veuillez soumettre l'extrait de l'émission ainsi que la citation que vous souhaitez que je corrige.",
 "Il semble que vous n'ayez pas fourni de texte ou de citation. Veuillez soumettre l'extrait de l'émission ainsi que la citation que vous souhaitez conserver, et je me ferai un plaisir de vous aider à corriger le texte.",
 "

In [4]:
df_claims.head()

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
67,f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c...,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
87,e57e39657004072e5f998d19e6a8a952ee8f35923b9072...,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
251,5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5c...,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative
259,53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...,2023-09-06 08:38:00,je suis même allé plus loin je vais essayer de...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences"", ""ressou...","[{""keyword"": ""\u00e9coresponsable"", ""timestamp...",2,614,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative
313,6c3bb9299d31332c644b54832399976117d28776c9e22c...,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative


In [33]:
pprint.pprint(contradictions.iloc[0].text)

('la conserve de trois cent quatre-vingt-cinq grammes c six ht soit deux euros '
 'vingt trois le kilo délicieuse au égale délicieux pour votre santé bougez '
 "plus a l'intention de notre très chère clientes propriétaire de la toyota "
 'ago crosses vous pouvez la déplacer vers les caméras de surveillance parce '
 "que j'aimerais bien admirer de plus près sur les écrans merci d'avance "
 'découvrez la toyota yaris cross le crossover urbain à partir de cent '
 'dix-neuf euros par mois entretien clé toyota yaris cross dynamique lld '
 'trente-sept mois trente mille km premier loyer premier trois loyer mille '
 'trois neuf cent mille soixante neuf euros cent réservée soixante-dix aux '
 'euros particuliers réservée aux particuliers sous conditions de reprise et '
 "commander jusqu'au vingt-cinq septembre détaille sur toyota point fr portes "
 'ouvertes ce week-end pensez à covoiturer écouter europe un partout dès '
 'maintenant en dab plus entre paris et marseille sans changer de fréquen

In [34]:
contradictions.iloc[0].text

"la conserve de trois cent quatre-vingt-cinq grammes c six ht soit deux euros vingt trois le kilo délicieuse au égale délicieux pour votre santé bougez plus a l'intention de notre très chère clientes propriétaire de la toyota ago crosses vous pouvez la déplacer vers les caméras de surveillance parce que j'aimerais bien admirer de plus près sur les écrans merci d'avance découvrez la toyota yaris cross le crossover urbain à partir de cent dix-neuf euros par mois entretien clé toyota yaris cross dynamique lld trente-sept mois trente mille km premier loyer premier trois loyer mille trois neuf cent mille soixante neuf euros cent réservée soixante-dix aux euros particuliers réservée aux particuliers sous conditions de reprise et commander jusqu'au vingt-cinq septembre détaille sur toyota point fr portes ouvertes ce week-end pensez à covoiturer écouter europe un partout dès maintenant en dab plus entre paris et marseille sans changer de fréquence et bientôt dans toute la france europe un tout

In [35]:
contradictions.iloc[0].quote

'Les déplacements en voiture sont la seule solution viable pour la mobilité.'

In [36]:
contradictions.iloc[0].quote in contradictions.iloc[0].text

False

In [37]:
"la conserve de trois cent quatre-vingt-cinq grammes" in contradictions.iloc[0].text

True

In [ ]:
contradictions["quote_in_text"] = contradictions.apply(lambda row: row["quote"] in row["text"], axis=1)
# les quote ne sont pas toujours dans les textes -> mesure de similarité entre claim et quote ? 
contradictions.groupby('quote_in_text').count()['id']

quote_in_text
False    180
True      98
Name: id, dtype: int64

In [48]:
pprint.pprint(contradictions[contradictions["quote_in_text"]==True].iloc[0].text)

('climatique le réchauffement collecte de grenoble de glaciologues on mesurait '
 "quand même à quatre mille trois cents mètres d'altitude qui ont fait un "
 "forage c'est-à-dire que sous le mont-blanc à sept mille trois cents mètres "
 'cinq cents mètres plus bas et dans ce forage il a été mesurée soixante '
 'mètres de neige est en dessous donc à peu près voilà cent quatre-vingts '
 "maître de glace c'est à dire de la neige imperméable il a été il a été foré "
 'est en à quatre-vingt moins quatorze onze degrés ont à été soixante moins '
 "mètres onze de degrés profondeur aujourd'hui ça s'est réchauffée de deux "
 'degrés on est à moins neuf degrés on a quatre mille trois cents mètres à '
 'soixante mètres de profondeur dans la neige et donc il y a quand même un '
 "réchauffement de ses effrois d'altitude et donc le loup blanc on n'a pas de "
 'thermomètre en permanence planté dans le mont-blanc et si on est à moins '
 "quinze degrés température moyenne peut être que aujourd'hui on es

In [47]:
contradictions[contradictions["quote_in_text"]==True].iloc[0].quote

'on ne peut pas faire le lien directement avec le réchauffement'

In [29]:
contradictions.iloc[0].values

array(['fcedb246b0d5b4e301404f2bb02fdec5f861a3822cbc5528b56eb5d144cd3eca',
       Timestamp('2023-09-14 18:14:00'),
       "la conserve de trois cent quatre-vingt-cinq grammes c six ht soit deux euros vingt trois le kilo délicieuse au égale délicieux pour votre santé bougez plus a l'intention de notre très chère clientes propriétaire de la toyota ago crosses vous pouvez la déplacer vers les caméras de surveillance parce que j'aimerais bien admirer de plus près sur les écrans merci d'avance découvrez la toyota yaris cross le crossover urbain à partir de cent dix-neuf euros par mois entretien clé toyota yaris cross dynamique lld trente-sept mois trente mille km premier loyer premier trois loyer mille trois neuf cent mille soixante neuf euros cent réservée soixante-dix aux euros particuliers réservée aux particuliers sous conditions de reprise et commander jusqu'au vingt-cinq septembre détaille sur toyota point fr portes ouvertes ce week-end pensez à covoiturer écouter europe un partout d

In [3]:
system_prompt = f"""
Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. 
Je vais te donner des claims sur des thématiques environnementales extraits de retranscriptions d'un flux TV ou Radio à factchecker. 
Pour chaque claim, donne une catégorisation de cette allégation suivant la classification qui suit.
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois le champ suivant : "cards_label_pred"

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu. Cette catégorie soutient que le changement climatique est inexistant ou largement exagéré. Elle suggère que des facteurs naturels — tels que le froid, la préservation de la glace et le refroidissement des océans — prouvent que le réchauffement n'est pas en cours. De plus, elle implique que les prévisions de changements climatiques sévères, y compris la montée du niveau de la mer et les phénomènes météorologiques extrêmes, sont exagérées ou inventées. Certaines affirmations soutiennent que la terminologie climatique a été manipulée (par exemple, en passant de "réchauffement climatique" à "changement climatique") pour alarmer le public sans raison.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique. Cette catégorie avance que les activités humaines, en particulier les émissions de gaz à effet de serre, ne sont pas responsables du réchauffement observé. Elle soutient que les cycles naturels, l'activité solaire ou d'autres facteurs non humains sont à l'origine des changements climatiques. Les affirmations de cette catégorie minimisent souvent le rôle du CO2, ou soulignent que les changements climatiques passés se sont produits sans intervention humaine, suggérant que le réchauffement actuel est également naturel.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais. Cette catégorie reconnaît qu'un certain réchauffement pourrait se produire, mais affirme qu'il pose peu de risques. Les arguments de cette catégorie suggèrent que la sensibilité du climat au CO2 est faible, ce qui limiterait le réchauffement. Les partisans prétendent que l'environnement, les espèces et la santé humaine seront largement épargnés, car les plantes bénéficient de l'augmentation du CO2, et que de nombreuses espèces, comme les ours polaires, peuvent s'adapter. Dans l'ensemble, elle suggère que les impacts climatiques seront mineurs et gérables.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas. Cette catégorie s'oppose aux politiques climatiques proposées, en affirmant qu'elles seront nuisibles économiquement ou socialement, inefficaces ou irréalistes. Elle soutient que les politiques de réduction des émissions augmenteront les coûts, affaibliront la sécurité nationale et restreindront les libertés individuelles sans bénéfices climatiques significatifs. D'autres affirmations proposent que les solutions "vertes" sont peu fiables, que l'énergie propre est non éprouvée, ou que les combustibles fossiles sont abondants et essentiels pour l'avenir. Essentiellement, elle soutient que les politiques climatiques sont trop coûteuses, difficiles ou inefficaces pour être poursuivies.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables. Cette catégorie attaque la crédibilité de la science climatique et du mouvement pour le climat, affirmant qu'ils sont non fiables, biaisés ou idéologiquement orientés. Elle remet en question le consensus scientifique sur le changement climatique, suggérant que les données, modèles et méthodes sont défectueux. De plus, elle présente le mouvement climatique comme alarmiste, biaisé ou faisant partie d'une conspiration plus large, motivée par des intentions politiques ou religieuses plutôt qu'une véritable préoccupation. Ce cadre cherche à miner la confiance dans la science climatique et dans ceux qui plaident pour l'action.
</catégories prédéfinies>
    """

encoding = tiktoken.encoding_for_model(model)
prompt_tokens = len(encoding.encode(system_prompt))

async def classify_claim(
        text: str, system_prompt: str = system_prompt, prompt_tokens: str = prompt_tokens, tpm_limit: int = 2e6
    ):
    text_tokens = len(encoding.encode(text))
    total_tokens = prompt_tokens + text_tokens
    wait_time_s = 60 * total_tokens / tpm_limit
    await asyncio.sleep(wait_time_s)
    
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

async def run_classification(texts: list[str]):
    semaphore = asyncio.Semaphore(50)
    
    async def bounded_classify_claim(text):
        async with semaphore:
            return await classify_claim(text)
    
    classification = await tqdm.gather(*[bounded_classify_claim(text) for text in texts])
    return classification

In [ ]:
classification = await run_classification(df_claims["claim"])
cards_label_preds = []

for item in classification:
    # Remove ```json and ``` 
    cleaned_item = re.sub(r'```json\n|```', '', item)

    # Load as JSON dictionary
    try:
        parsed_item = json.loads(cleaned_item)
        cards_label_preds.append(parsed_item["cards_label_pred"])
    except json.JSONDecodeError:
        print(f"Parsing error: {cleaned_item}")

cards_label_preds
df_claims["cards_label_pred"] = cards_label_preds

df_exploded["cards_label_pred"] = ''
df_exploded.loc[df_claims.index, "cards_label_pred"] = df_claims["cards_label_pred"]

def add_key(row):
    try:
        return {**row['claims'], 'cards_label_pred': row['cards_label_pred']}
    except:
        return row['claims']

df_exploded['claims'] = df_exploded.apply(lambda row: add_key(row), axis=1)
df_exploded = df_exploded.drop(columns=['cards_label_pred'])
df_grouped = df_exploded.groupby('id', as_index=False).agg({'claims': list})

def isnan_(row):
    try:
        return math.isnan(row.claims[0])
    except:
        return False

df.loc[~ df_grouped.apply(lambda row: isnan_(row), axis=1), 'claims'] = df_grouped.loc[~ df_grouped.apply(lambda row: isnan_(row), axis=1), 'claims']
# df.to_csv("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024_with_cards.csv")

In [69]:
df_cards = pd.read_csv("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024_with_cards.csv")
df_cards.drop(columns={'Unnamed: 0'}, inplace=True)

df_cards['claims'] = df_cards['claims'].apply(lambda x: re.sub(r'}\s*{', '}, {', x))
df_cards['claims'] = df_cards['claims'].apply(lambda x: ast.literal_eval(x))

In [60]:
def find_claim(row, claim):
    try:
        return row.claims[0]['claim'] == claim
    except:
        return False

df[df.apply(lambda row: find_claim(row, 'le giec est transhumaniste'), axis=1)].id.values

array(['f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c796a1a8ddba78393da'],
      dtype=object)

In [84]:
print(df['claims'].apply(lambda x: len(x)).unique())
print(df_cards['claims'].apply(lambda x: len(x)).unique())

[ 1  4  2  0  3  5 19  7  6 17  8  9]
[ 1  5  4  3  2  0  7  6 17  9 19  8]


In [86]:
df['claims'].iloc[1]

array([{'analysis': "L'allégation semble être fondée sur des données d'économies d'eau. Néanmoins, cela dépend de l'adoption généralisée de la technologie et des conditions de l'utilisation de l'eau dans les campings. Il serait prudent d'explorer si ces chiffres sont corroborés par des études ou des données réelles.", 'claim': "La facture s'allège de manière non négligeable avec cinquante pour cent d'économie d'eau atteignant parfois les six mille euros annuel.", 'context': "Dans une discussion sur une nouvelle innovation dans les campings qui vise à économiser de l'eau, il est mentionné que cette technologie permet de réduire la consommation d'eau et les coûts associés.", 'contradiction': 'N/A', 'disinformation_category': 'facts', 'disinformation_score': 'low', 'pro_anti': 'pro-écologie', 'quote': "La facture s'allège de manière non négligeable avec cinquante pour cent d'économie d'eau atteignant parfois les six mille euros annuel.", 'speaker': 'narrative'},
       {'analysis': "Cet é

In [89]:
df.iloc[1].id == df_cards.iloc[1].id

True

In [87]:
df_cards['claims'].iloc[1]

[{'analysis': "Cette allégation semble plausible et correspond à des études précédentes sur l'impact environnemental de l'industrie textile. Il est bien documenté que la production de coton est extrêmement consommatrice d'eau. Il semble donc que cela ne soit pas de la désinformation.",
  'claim': "Il faut environ 18 900 litres d'eau pour fabriquer un jean en coton.",
  'context': "Dans un discours abordant l'impact environnemental de l'habillement, il a été mentionné qu'un jean nécessite une quantité importante d'eau pour sa fabrication.",
  'contradiction': 'N/A',
  'disinformation_category': 'facts',
  'disinformation_score': 'very low',
  'pro_anti': 'pro-écologie',
  'quote': "Vous savez combien de litres d'eau il faut claude pour fabriquer un belot jean baudelot bawe le coton sur dix-huit cent neuf mille.",
  'speaker': 'consensus',
  'cards_label_pred': ''}]

In [90]:
df_cards.shape

(39131, 12)

In [91]:
df.shape

(39131, 12)

In [85]:
df['claims'].apply(lambda x: len(x)) == df_cards['claims'].apply(lambda x: len(x))

0         True
1        False
2        False
3         True
4        False
         ...  
39126    False
39127     True
39128     True
39129    False
39130    False
Name: claims, Length: 39131, dtype: bool

In [63]:
df[df.apply(lambda row: find_claim(row, 'des textes sur les énergies renouvelables'), axis=1)].id.values

array(['f632de53bfe36f76d420ff1295d2d9a2a18fc9cb8e560daf4d413246bd55f816'],
      dtype=object)

In [67]:
import pprint
pprint.pprint(df[df.id == 'f632de53bfe36f76d420ff1295d2d9a2a18fc9cb8e560daf4d413246bd55f816'].claims.values)

array([array([{'analysis': "L'allégation fait référence à une législation existante sur les énergies renouvelables. Il n'y a pas de désinformation ici, mais il est important de vérifier le contenu spécifique des textes pour comprendre leur impact et leur portée.", 'claim': 'des textes sur les énergies renouvelables', 'context': 'Le rapporteur mentionne les lignes adoptées concernant les énergies renouvelables tout en les intégrant dans un débat plus large sur différents textes de lois, y compris sur le nucléaire et le militaire.', 'contradiction': 'N/A', 'disinformation_category': 'facts', 'disinformation_score': 'very low', 'pro_anti': 'pro-écologie', 'quote': 'des textes sur les énergies renouvelables', 'speaker': 'consensus'},
              {'analysis': 'Cette affirmation aborde une réalité des politiques énergétiques actuelles sans indiquer de fausse information. Cela mérite cependant une vérification des détails des textes adoptés.', 'claim': 'adopter des textes sur le nucléaire',

In [72]:
pprint.pprint(df_cards[df_cards.id == 'f632de53bfe36f76d420ff1295d2d9a2a18fc9cb8e560daf4d413246bd55f816'].claims.values)

array(['[{\'analysis\': "L\'allégation fait référence à une législation existante sur les énergies renouvelables. Il n\'y a pas de désinformation ici, mais il est important de vérifier le contenu spécifique des textes pour comprendre leur impact et leur portée.", \'claim\': \'des textes sur les énergies renouvelables\', \'context\': \'Le rapporteur mentionne les lignes adoptées concernant les énergies renouvelables tout en les intégrant dans un débat plus large sur différents textes de lois, y compris sur le nucléaire et le militaire.\', \'contradiction\': \'N/A\', \'disinformation_category\': \'facts\', \'disinformation_score\': \'very low\', \'pro_anti\': \'pro-écologie\', \'quote\': \'des textes sur les énergies renouvelables\', \'speaker\': \'consensus\'}\n {\'analysis\': \'Cette affirmation aborde une réalité des politiques énergétiques actuelles sans indiquer de fausse information. Cela mérite cependant une vérification des détails des textes adoptés.\', \'claim\': \'adopter des 